In [1]:
import pandas as pd
import numpy as np
from numpy import exp,pi,sin,cos,sinh,sqrt,roots
#from scipy.integrate import tplquad,quad,dblquad
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from Parameters import Const
from MEWs_Funcs import Spec_q,Spec_w,Derivatives,Curvarure
from Save_Funcs import Save_to_Files

font_names = [f.name for f in fm.fontManager.ttflist]
#print(font_names)

# Parameters

In [2]:
x = 0.18
mb,hp,g,ge,Ka,ro,a0,c11,c12,c44,M0,A,b2,al = Const(x)

print('c11 =',c11,'c12 =',c12,'c44 =',c44,'ro =',ro)
print('a =',a0,'M0 =',M0,'A =',A,'b2 =',b2)

c11 = 2086220397359.9995 c12 = 1672968839999.9998 c44 = 1252638503999.9998 ro = 7.5905
a = 2.8999992719999997e-08 M0 = 1358.315664 A = 8.9092308e-07 b2 = -54979528.84000003


# Spectrum, Amplification and Curvature; Derivatives

In [3]:
t_array = np.linspace(1.5708E-4,pi/2,100) #t_array = np.ones(1)*0.30473448739821
fi = pi*0.17

H00 = 3e3
w = ge*(2*Ka/M0 + H00)
H0 = 2940
H = 2*Ka/M0 + H0

qr,Ar, Kc = np.zeros([t_array.size,4], dtype=np.complex128), np.zeros([t_array.size,4]), np.zeros([t_array.size,4], dtype=np.complex128)
qt,qf = np.zeros([t_array.size,4], dtype=np.complex128), np.zeros([t_array.size,4], dtype=np.complex128)
qtt,qtf,qff = np.zeros([t_array.size,4], dtype=np.complex128), np.zeros([t_array.size,4], dtype=np.complex128), np.zeros([t_array.size,4], dtype=np.complex128)
I1,II1 = np.zeros([t_array.size,4]), np.zeros([t_array.size,4], dtype=np.complex128)

for t in t_array:
    q = []
    for qq in Spec_q(w,t,fi,x,H):
        if np.real(qq) >= 0:
            q.append(qq)
    
    f = np.sort(q)[::-1]
    q = f[np.real(f>0)]

    ii, =  np.where(np.isclose(t_array, t))
    ii = ii.item()

    for mode in range(4):
        qr[ii,mode] = q[mode]
        Kc[ii,mode] = Curvarure(w,q[mode],t,fi,x,H)[0]/a0**2
        Ar[ii,mode] = Curvarure(w,q[mode],t,fi,x,H)[1]
        qt[ii,mode] = Derivatives(w,q[mode],t,fi,x,H)[0]
        qf[ii,mode] = Derivatives(w,q[mode],t,fi,x,H)[1]
        qtt[ii,mode] = Derivatives(w,q[mode],t,fi,x,H)[2]
        qff[ii,mode] = Derivatives(w,q[mode],t,fi,x,H)[3]
        qtf[ii,mode] = Derivatives(w,q[mode],t,fi,x,H)[4]
        I1[ii,mode] = Derivatives(w,q[mode],t,fi,x,H)[5]*a0**4
        II1[ii,mode] = Derivatives(w,q[mode],t,fi,x,H)[6]*a0**2

Save data to files

In [4]:
Save_to_Files(t_array,Ar,Kc,qt,qf,qtt,qff,qtf,I1,II1)

# Plotting

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8, 8), subplot_kw={'projection': 'polar'})
ax[0].plot(t_array, np.real(qr[:,0]),t_array, np.real(qr[:,1]),t_array, np.real(qr[:,2]),t_array, np.real(qr[:,3]))
ax[0].set_thetalim([0,pi/2])
ax[0].set_thetagrids([0, 15, 30, 45, 60, 75, 90])
ax[0].grid(True)
ax[1].plot(t_array, np.imag(qr[:,0]),t_array, np.imag(qr[:,1]),t_array, np.imag(qr[:,2]),t_array, np.imag(qr[:,3]))
ax[1].set_thetalim([0,pi/2])
ax[1].set_thetagrids([0, 15, 30, 45, 60, 75, 90])
ax[1].grid(True)
plt.show()

fig, ax = plt.subplots(1,2, figsize=(14, 5))
ax[0].plot(t_array, np.real(Kc[:,0]),t_array, np.real(Kc[:,1]),t_array, np.real(Kc[:,2]),t_array, np.real(Kc[:,3]))
ax[0].set(xlabel='theta', ylabel='Re(K)')
ax[0].yaxis.get_major_formatter().set_useOffset(True)
ax[0].grid()
ax[1].plot(t_array, np.imag(Kc[:,0]),t_array, np.imag(Kc[:,1]),t_array, np.imag(Kc[:,2]),t_array, np.imag(Kc[:,3]))
ax[1].set(xlabel='theta', ylabel='Im(K)')
ax[1].grid()
plt.show()

fig, ax = plt.subplots(1,1, figsize=(14, 5))
ax.plot(t_array, Ar[:,0],t_array, Ar[:,1],t_array, Ar[:,2],t_array, Ar[:,3])
ax.set(xlabel='theta', ylabel='Ampl_Factor')
ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1)) 
ax.yaxis.set_minor_locator (mpl.ticker.MultipleLocator(0.25))
ax.set_yscale('log')
ax.grid()
mpl.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.linewidth'] = 1.5
plt.show()